In [ ]:
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import os
import json

# ===== 1️⃣ 生成多轮回答 =====
def roll_llm_responses(problem, n_roll=5):
    """
    对同一个文本问题 roll 出多组回答（适用于纯文本 Qwen）
    """
    print(f"\n🧩 当前问题: {problem[:100]}...")
    
    # ✅ 构造多轮 messages
    messages = [
        {"role": "system", "content": "You are Qwen, a helpful assistant developed by Alibaba Cloud."},
        {"role": "user", "content": (
            f"{problem}\n\n"
            "You should first think step by step. "
            "The reasoning process MUST be enclosed in <think>...</think> tags. "
            "Then give your final answer in \\boxed{{}} format. "
            "Finally, output a confidence score (1–10) in <confidence></confidence> tags."
        )}
    ]

    # ✅ 转换为 chat 模板输入
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generations = []
    for i in range(n_roll):
        print(f"生成第 {i+1}/{n_roll} 个回答...")
        output_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
        )

        # 去掉 prompt token 部分
        output_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(model_inputs.input_ids, output_ids)
        ]
        resp = tokenizer.batch_decode(output_ids_trimmed, skip_special_tokens=True)[0]
        print(f"🧠 Roll {i+1} 输出摘要: {resp[:200]}...\n")
        generations.append(resp)
    return generations


# ===== 2️⃣ 计算置信度区分 =====
def test_llm_confidence_separation(row, n_roll=5, save_dir="./logs_llm"):
    os.makedirs(save_dir, exist_ok=True)

    gt = row["answer"]
    problem = row["problem"]

    print(f"\n===================== 🧩 测试样本 =====================")
    print(f"题目: {problem[:100]}...")
    print(f"标准答案: {gt}")
    print("========================================================")

    responses = roll_llm_responses(problem, n_roll=n_roll)
    reward_inputs = [{"response": r, "ground_truth": gt} for r in responses]
    scores = compute_score(reward_inputs)  # ✅ 使用你已有的 reward 函数

    # 分类统计
    conf_correct, conf_wrong, record_each = [], [], []
    for idx, s in enumerate(scores):
        acc = s["accuracy"]
        conf = s["confidence"]
        rec = {
            "roll_id": idx + 1,
            "response": reward_inputs[idx]["response"],
            "ground_truth": gt,
            "confidence": conf,
            "accuracy": acc,
        }
        record_each.append(rec)
        if acc == 1.0:
            conf_correct.append(conf)
        else:
            conf_wrong.append(conf)

    # 计算统计
    mean_correct = np.mean(conf_correct) if conf_correct else np.nan
    mean_wrong = np.mean(conf_wrong) if conf_wrong else np.nan
    delta = mean_correct - mean_wrong if not np.isnan(mean_correct) and not np.isnan(mean_wrong) else np.nan

    print(f"\n✅ 正确样本数: {len(conf_correct)} | ❌ 错误样本数: {len(conf_wrong)}")
    print(f"✅ 正确样本置信度均值: {mean_correct:.3f}")
    print(f"❌ 错误样本置信度均值: {mean_wrong:.3f}")
    print(f"Δ (差值) = {delta:.3f}")

    if abs(delta) < 0.05 or np.isnan(delta):
        print("⚠️ 差异极小或无法比较 → 可能是过拟合到 prompt。")
    else:
        print("✅ 置信度区分明显 → 模型可能基于 reasoning 学习信心度。")

    # 保存 JSON 日志
    record = {
        "problem": problem,
        "ground_truth": gt,
        "n_roll": n_roll,
        "mean_conf_correct": mean_correct,
        "mean_conf_wrong": mean_wrong,
        "delta": delta,
        "each_roll": record_each,
    }

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{save_dir}/sample_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(record, f, ensure_ascii=False, indent=2)

    print(f"📁 已保存记录: {filename}\n")
    return record


# ===== 3️⃣ 批量运行 =====
all_records = []
for i in range(len(llm_df)):  # 遍历整个 LLM 数据集
    try:
        res = test_llm_confidence_separation(llm_df.iloc[i], n_roll=10)
        all_records.append(res)
    except Exception as e:
        print(f"❌ 样本 {i} 出错: {e}")

# 汇总 DataFrame
summary_rows = [
    {
        "problem": r["problem"][:60],
        "mean_conf_correct": r["mean_conf_correct"],
        "mean_conf_wrong": r["mean_conf_wrong"],
        "delta": r["delta"],
        "n_roll": r["n_roll"],
    }
    for r in all_records if r
]
result_df = pd.DataFrame(summary_rows)
print("\n=== 📊 汇总统计 ===")
print(result_df)

result_df.to_csv("confidence_summary_llm.csv", index=False)
print("✅ 已保存 CSV 文件：confidence_summary_llm.csv")


In [ ]:
import os
import json
import numpy as np
from sklearn.metrics import roc_auc_score

def compute_ece(confidences, accuracies, n_bins=10):
    """Expected Calibration Error"""
    bins = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        mask = (confidences >= bins[i]) & (confidences < bins[i + 1])
        if np.any(mask):
            acc_bin = np.mean(accuracies[mask])
            conf_bin = np.mean(confidences[mask])
            ece += np.abs(acc_bin - conf_bin) * np.mean(mask)
    return ece

def analyze_log_folder(folder="log_llm"):
    all_confs, all_accs = [], []

    results = []
    for filename in os.listdir(folder):
        if not filename.endswith(".json"):
            continue
        with open(os.path.join(folder, filename), "r", encoding="utf-8") as f:
            data = json.load(f)
        
        rolls = data.get("each_roll", [])
        confs = np.array([r["confidence"] for r in rolls if r.get("confidence") is not None])
        accs  = np.array([r["accuracy"]   for r in rolls if r.get("accuracy")   is not None])
        
        if len(confs) == 0: continue

        mean_conf_correct = confs[accs == 1].mean() if np.any(accs == 1) else np.nan
        mean_conf_wrong   = confs[accs == 0].mean() if np.any(accs == 0) else np.nan
        delta = mean_conf_correct - mean_conf_wrong if not np.isnan(mean_conf_correct) and not np.isnan(mean_conf_wrong) else np.nan
        
        # 汇总全局统计
        all_confs.extend(confs)
        all_accs.extend(accs)

        results.append({
            "file": filename,
            "mean_conf_correct": mean_conf_correct,
            "mean_conf_wrong": mean_conf_wrong,
            "delta": delta,
            "acc_rate": np.mean(accs)
        })

    # 全局指标
    all_confs = np.array(all_confs)
    all_accs = np.array(all_accs)

    global_stats = {
        "global_mean_conf_correct": all_confs[all_accs == 1].mean(),
        "global_mean_conf_wrong": all_confs[all_accs == 0].mean(),
        "global_delta": all_confs[all_accs == 1].mean() - all_confs[all_accs == 0].mean(),
        "global_auc": roc_auc_score(all_accs, all_confs) if len(np.unique(all_accs)) > 1 else np.nan,
        "global_ece": compute_ece(all_confs, all_accs),
        "global_brier": np.mean((all_confs - all_accs) ** 2),
        "global_acc": np.mean(all_accs)
    }

    return results, global_stats

results, global_stats = analyze_log_folder("log_llm")

print("=== 每个文件的统计 ===")
for r in results:
    print(r)

print("\n=== 全局统计 ===")
for k, v in global_stats.items():
    print(f"{k}: {v:.4f}")
